<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Meta_Llama_3_8B_for_MEDAL_EVALDATA_PONEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://llama.meta.com/docs/how-to-guides/fine-tuning/


https://medium.com/@frankmorales_91352/fine-tuning-meta-llama-3-8b-with-medal-a-refined-approach-for-enhanced-medical-language-b924d226b09d

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

! pip install trl==0.8.6 -q

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [ ]:
from huggingface_hub import login

login(
  token=access_token_write, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

In [ ]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
    EarlyStoppingCallback
)


from trl import SFTTrainer

In [ ]:
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.001
)


In [ ]:
# set device
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.__version__

'2.6.0+cu124'

In [ ]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.11.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Wed May  7 10:23:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   3

In [ ]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

{'prompt': '<prompt text>', 'completion': '<ideal generated text>'}

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/train_dataset.json", split="train")

In [ ]:
dataset

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 3000000
})

In [ ]:
print(dataset[6]['text'])

to quantify the amount of CL NO harvested in modified RND


In [ ]:
print(dataset[6]['label'])

['radical neck dissection']


In [ ]:
dataset_test = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

In [ ]:
dataset_test

Dataset({
    features: ['abstract_id', 'text', 'location', 'label'],
    num_rows: 1000000
})

In [ ]:
import re

def dataset_to_finetuning(dataset,nrec):

    question=dataset['text']
    answer_entry=dataset['label']


    !rm -rf /content/question_answering.txt
    filename='/content/question_answering.txt'


  # python object to be appended
    for n in range(nrec):
        #print(answer[n])
        if answer_entry[n] == None:
           answer_entry[n] = 'Not possible to get or use'
        string = answer_entry[n]

        str_question=question[n]
        question_final= re.sub("\n", "", str_question)

        answer = re.sub("\[", "", str(string))
        answer = re.sub("\]", "", str(answer))
        answer0=answer[1:len(answer)-1]

        if len(answer0)==0:
            answer='Not possible to get or use'

        #print(answer0)


        text='<s>[INST] %s [/INST] %s </s>'%(question_final,answer0)
        del string,answer0
        #print(text)
        with open(filename, 'a') as f:
            f.write(text + '\n')
        f.close()

    text0 = load_dataset("text", data_files="/content/question_answering.txt", split="train")

    dataset_final_Question=dataset['text'][0:nrec]
    dataset_final_Answer=dataset['label'][0:nrec]
    dataset_final_text=text0[0:nrec]['text']

    import pandas as pd

    datasetF = pd.DataFrame() # Create an empty DataFrame
    datasetF['Question'] = dataset_final_Question
    datasetF['Answer'] = dataset_final_Answer
    datasetF['text'] = dataset_final_text


    return datasetF

In [ ]:
datasetF=dataset_to_finetuning(dataset,500000)

In [ ]:
datasetF['Question'][6]

'to quantify the amount of CL NO harvested in modified RND'

In [ ]:
datasetF['Answer'][6]

['radical neck dissection']

In [ ]:
datasetF['text'][6]

'<s>[INST] to quantify the amount of CL NO harvested in modified RND [/INST] radical neck dissection </s>'

In [ ]:
datasetF

In [ ]:
datasetF_test=dataset_to_finetuning(dataset_test,20000)

In [ ]:
datasetF_test

In [ ]:
datasetF_test['Question'][6]

'while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in summar

In [ ]:
datasetF_test['Answer'][6]

['antral follicle count']

In [ ]:
datasetF_test['text'][6]

'<s>[INST] while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
#model_id = "mistralai/Mistral-7B-Instruct-v0.1" #01 march 2024 AND 10/03/2024

model_id = "meta-llama/Meta-Llama-3-8B" #22 MAY 2024

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id


In [ ]:
print(model)

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
        lora_alpha=64,
        lora_dropout=0.05,
        r=128,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="/content/gdrive/MyDrive/model/07MAY2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata-epochs1",

    #num_train_epochs=3,                     # number of training epochs
    #num_train_epochs=0.3,                    # number of training epochs for POC
    num_train_epochs=1,
    per_device_train_batch_size=1,          # batch size per device during training
    #2
    gradient_accumulation_steps=8,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    #gradient_checkpointing_kwargs={"use_reentrant": True},
    optim="adamw_torch_fused",              # use fused adamw optimizer

    warmup_ratio=0.15,
    weight_decay=0.05,

    logging_steps=100,                       # log every 10 steps
    learning_rate=5e-5,                     # learning rate, based on QLoRA paper # i used in the first model

    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=1.0,                      # max gradient norm based on QLoRA paper


    #lr_scheduler_type="constant",           # use constant learning rate scheduler
    lr_scheduler_type="cosine",            # lr_scheduler_type="cosine" (Cosine Annealing Learning Rate)

    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs={"use_reentrant": True},

    load_best_model_at_end=True,
    logging_dir="/content/gdrive/MyDrive/model/07MAY2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata-epoch1/logs",

    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    metric_for_best_model = "loss",
)

In [ ]:
!pip install trl -q

In [ ]:
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint
#torch.utils.checkpoint.checkpoint(use_reentrant=False)

https://huggingface.co/docs/trl/main/en/sft_trainer

In [ ]:
from trl import SFTTrainer
import datasets
#from datasets import load_metric
from warnings import filterwarnings
filterwarnings('ignore')

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)
datasetF_hf_test = datasets.Dataset.from_pandas(datasetF_test)

#max_seq_length = 3072 # max sequence length for model and packing of the dataset
max_seq_length = 1024

# Explicitly set the device using torch.device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Add a padding token to your tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Or, add a new padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

#metric = load_metric("squad")  # Assuming you're evaluating on SQuAD dataset
#def compute_metrics(eval_pred):
#    logits, labels = eval_pred
#    predictions = logits.argmax(axis=-1)
#    return metric.compute(predictions=predictions, references=labels)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=datasetF_hf,
    dataset_text_field="text", ### added for the summarization dataset
    eval_dataset=datasetF_hf_test,

    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    #processing_class=tokenizer,
    packing=True,
    #callbacks=[early_stopping_callback],

    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py

In [ ]:
from transformers import EarlyStoppingCallback


trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=5))


# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

In [ ]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

## Test Model and run Inference

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = "/content/gdrive/MyDrive/model/07MAY2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata-epochs1"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16,
  attn_implementation="flash_attention_2",
  quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Let’s load our test dataset try to generate an instruction.

In [ ]:
print(model.peft_config)

{'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='meta-llama/Meta-Llama-3-8B', revision=None, inference_mode=True, r=128, target_modules={'q_proj', 'up_proj', 'v_proj', 'gate_proj', 'down_proj', 'o_proj', 'k_proj'}, exclude_modules=None, lora_alpha=64, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")
nrec= randint(0, len(eval_dataset))
nrec=6

# Test on sample
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")  # Add device_map
prompt =  eval_dataset[nrec]['text']


outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.9,
                                  top_k=30, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
ganswer

'antral follicle count </s><s>[INST] the aim of this T0 was to evaluate the effect of a single dose of the nonsteroidal antiinflammatory drug nsaids on the pharmacokinetics of the antiepileptic drug carbamazepine cbz in healthy volunteers a single dose of mg of cbz was administered to healthy volunteers in a randomized crossover design T3 a single dose of mg of nsaids or PL the pharmacokinetics of cbz were determined by measuring the plasma concentration of cbz and its metabolites carbamazepine epoxide cbzepoxide and hydro'

In [ ]:
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
oanswer=str(eval_dataset[nrec]['label'])
oanswer=oanswer[2:len(oanswer)-2]
print(f"Original Answer:\n{oanswer}")
print()
ganswer=outputs[0]['generated_text'][len(prompt)+9:].strip()
qc=str(ganswer).find('[INST]')
ganswer=ganswer[0:qc-7]
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0]
qc=str(ganswer).find('[/INST]')
if qc>0:
  ganswer=ganswer[qc+8:len(ganswer)]
print(f"Generated Answer:\n{ganswer}")
print()
if ganswer == oanswer:
  print("Match")
else:
  print("NO Match")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [ ]:
nrec=6
print(f"Query:\n{eval_dataset[nrec]['text']}")
print()
print(f"Original Answer:\n{eval_dataset[nrec]['label']}")
print()
print(f"Full Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Query:
while diminished ovarian reserve dor predicts decreased ovarian response to stimulation it does not necessarily foretell about the fecundity cycle according to bolognas criteria laid down by the european society of human reproduction and embryology old age abnormal ovarian reserve tests such as AFC afc and antimullerian hormone amh as well as prior suboptimal response to stimulation are the main AF representing dor unfavorable response to maximal stimulation on two previous occasions may also represent dor among the ovarian reserve tests amh and afc are the most predictive values for dor AF which may give rise to dor include environmental factors autoimmune or metabolic disorders infections genetic abnormalities and iatrogenic causes such as smoking chemotherapy radiation and gynecologic surgeries besides studies have proposed endometriosis as a key contributor to dor and hence emphasized on its proper management to prevent additional damages leading to compromised fertility in 

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc=str(ganswer).find('[/INST]')
#print(qc)
if int(qc)<0:
    #print(f"Generated Answer:\n{ganswer}")
    ganswer=outputs[0]['generated_text'][len(prompt)+8:].strip()
else:
    ganswer=ganswer[qc:len(ganswer)]
ganswer

'[/INST] antral follicle count </s><s>[INST] the aim of this T0 was to evaluate the effect of a single dose of the nonsteroidal antiinflammatory drug nsaids on the pharmacokinetics of the antiepileptic drug carbamazepine cbz in healthy volunteers a single dose of mg of cbz was administered to healthy volunteers in a randomized crossover design T3 a single dose of mg of nsaids or PL the pharmacokinetics of cbz were determined by measuring the plasma concentration of cbz and its metabolites carbamazepine epoxide cbzepoxide and hydro'

In [ ]:
ganswer=outputs[0]['generated_text'][len(prompt):].strip()
qc0=str(ganswer).find('[INST]')
ganswer=str(ganswer)[0:qc0-8]
qc=str(ganswer).find('[/INST]')

if qc>=0:
  ganswer=ganswer[qc+8:len(ganswer)]

print(f"Generated Answer:\n{ganswer}")

Generated Answer:
antral follicle count


In [ ]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from transformers import pipeline
from datasets import load_dataset

# Load a pre-trained sentence embedding model
model_embedding = SentenceTransformer('all-mpnet-base-v2')

# Load your test dataset
eval_dataset = load_dataset("json", data_files="/content/gdrive/MyDrive/datasets/McGill-NLP/test_dataset.json", split="train")

# Create the generation pipeline
generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

def evaluate(sample):
    prompt = sample['text']
    outputs = generation_pipeline(prompt, max_new_tokens=128, do_sample=True, temperature=0.7,
                                  top_k=50, top_p=0.7, eos_token_id=tokenizer.eos_token_id,
                                  pad_token_id=tokenizer.eos_token_id)

    ganswer = outputs[0]['generated_text'][len(prompt):].strip()
    qc0 = str(ganswer).find('[/INST]')
    qc1 = str(ganswer).find('[INST]')
    ganswer = str(ganswer)[qc0 + 8:qc1 - 8]

    oanswer = str(sample['label'])
    oanswer = oanswer[2:len(oanswer) - 2]

    if len(oanswer) == 0:
        oanswer = 'Not possible to get or use'

    #print('\n\n')
    #print(f"Question: {prompt}")
    #print(f"Generated Answer: {ganswer}")
    #print(f"Original Answer: {oanswer}")

    # Generate sentence embeddings
    embedding1 = model_embedding.encode(ganswer, convert_to_tensor=True)
    embedding2 = model_embedding.encode(oanswer, convert_to_tensor=True)

    # Calculate cosine similarity
    cosine_sim = util.cos_sim(embedding1, embedding2).item()

    # Set a similarity threshold (adjust as needed)
    if cosine_sim >= 0.5:
        #print("Match")
        return 1
    else:
        return 0


success_rate = []
number_of_eval_samples = 10  # Adjust the number of samples as needed

# Evaluate the model
for n in tqdm(range(number_of_eval_samples)):
    s = eval_dataset[n]
    success_rate.append(evaluate(s))


In [ ]:
# Calculate accuracy
accuracy = sum(success_rate) / len(success_rate)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6


## Tensorbard

In [ ]:
%rm -rf /content/tensorbard
%mkdir -p /content/tensorbard
%cd /content/tensorbard
!git lfs install
!git clone https://huggingface.co/frankmorales2020/11APRIL2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata

/content/tensorbard
Git LFS initialized.
Cloning into '11APRIL2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (232/232), done.
remote: Total 235 (delta 86), reused 0 (delta 0), pack-reused 3 (from 1)
Receiving objects: 100% (235/235), 59.57 KiB | 19.86 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/tensorbard/07MAY2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata-epoch1/logs/

## AGENT

In [ ]:
!pip install google-adk -q

In [ ]:
!pip install nest_asyncio -q

In [ ]:
import nest_asyncio
import asyncio
import os
from google.adk.agents import Agent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


# Define the local path to your fine-tuned model
# This path is from your Google Drive mount in the Colab environment
FINE_TUNED_MODEL_PATH = "/content/gdrive/MyDrive/model/07MAY2025-Meta-Llama-3-8B-MEDAL-flash-attention-2-cosine-evaldata-epochs1"

# Load the model and tokenizer from the local path using transformers
# We assume this code is run in an environment where this path is accessible
try:
    # Determine the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(FINE_TUNED_MODEL_PATH)
    # Set padding token if it's not already set
    if tokenizer.pad_token is None:
         tokenizer.pad_token = tokenizer.eos_token
         if tokenizer.pad_token is None:
             # Fallback if EOS token is also not set
             tokenizer.add_special_tokens({'pad_token': '[PAD]'})


    # Load the model
    # Use torch_dtype=torch.bfloat16 if your GPU supports it for potentially faster inference
    model = AutoModelForCausalLM.from_pretrained(FINE_TUNED_MODEL_PATH, torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32).to(device)
    model.eval() # Set the model to evaluation mode

    print(f"Successfully loaded model and tokenizer from {FINE_TUNED_MODEL_PATH}")

except Exception as e:
    print(f"Error loading model or tokenizer from {FINE_TUNED_MODEL_PATH}. Please ensure the path is correct and the necessary libraries are installed.")
    print(f"Details: {e}")
    # Exit or handle the error appropriately if the model cannot be loaded
    model = None
    tokenizer = None


# Define a simple agent class inheriting from ADK's Agent
class LocalFineTunedMedicalAgent(Agent):
    def __init__(self, name: str, instruction: str, model, tokenizer):
        # Initialize the Agent with name and instruction, model is a string
        # Set model path instead of None as ADK expects a string
        super().__init__(name=name, instruction=instruction, model=FINE_TUNED_MODEL_PATH)
        self._local_model = model
        self._local_tokenizer = tokenizer
        self._device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # The core asynchronous method where the agent processes input
    async def _run_async_impl(self, context): # Remove type hint for InvocationContext
        # Access the user's query from the invocation context
        user_query = context.request.message.text

        # print(f"Agent '{self.name}' received query: {user_query}") # Uncomment for more detailed logging

        # --- Text Generation using the locally loaded transformers model ---
        try:
            # Prepare the prompt
            # You might want to format the prompt based on your model's
            # preferred input format (e.g., chat format like Llama 3)
            # For this example, we'll use a simple instruction format derived from your notebook
            # Assuming your model was fine-tuned with prompts like "[INST] instruction [/INST]"
            prompt = f"[INST] {self.instruction}\n{user_query} [/INST]"


            # Tokenize the input
            inputs = self._local_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(self._device)

            # Generate text
            with torch.no_grad(): # Disable gradient calculation for inference
                 outputs = self._local_model.generate(
                     **inputs,
                     max_new_tokens=256, # Limit the generated response length
                     num_return_sequences=1,
                     do_sample=True,
                     top_k=50,
                     top_p=0.95,
                     temperature=0.7,
                     # Add other generation parameters as needed
                     pad_token_id=self._local_tokenizer.pad_token_id # Ensure pad_token_id is set for generation
                 )

            # Decode the generated tokens
            generated_text_with_prompt = self._local_tokenizer.decode(outputs[0], skip_special_tokens=False)

            # Post-process the generated text to extract the agent's response
            # Based on the prompt format "[INST] ... [/INST] response", we want the part after [/INST]
            if "[/INST]" in generated_text_with_prompt:
                generated_text = generated_text_with_prompt.split("[/INST]", 1)[1].strip()
            else:
                 # Fallback if the expected format is not present
                 generated_text = generated_text_with_prompt.strip()

            # Further cleaning might be needed depending on model output (e.g., cutting off at a specific token or phrase)
            # If the model generates a new turn of the prompt, cut it off
            if "[INST]" in generated_text:
                 generated_text = generated_text.split("[INST]", 1)[0].strip()


        except Exception as e:
            generated_text = f"An error occurred during text generation: {e}"
            print(f"Error during text generation with transformers: {e}")

        # Yield the final response from the agent
        yield context.AgentResponse( # Use context.AgentResponse directly
            message=context.AgentMessage(text=generated_text) # Use context.AgentMessage directly
        )


In [ ]:
!pip show google-adk

In [ ]:
async def main():
    # Ensure model and tokenizer were loaded successfully
    if model is None or tokenizer is None:
        print("Model loading failed. Exiting.")
        return

    # 1. Instantiate the in-memory services for artifacts and sessions
    artifact_service = InMemoryArtifactService()
    session_service = InMemorySessionService()

    # 2. Create an instance of your local fine-tuned agent
    agent_name = "LocalFineTunedMedicalAssistant"
    agent_instruction = (
        "You are a highly specialized AI medical assistant, fine-tuned to provide information "
        "based on medical literature. Answer questions accurately and concisely, focusing on "
        "information related to medical concepts and terminology."
    )

    medical_agent = LocalFineTunedMedicalAgent(
        name=agent_name,
        instruction=agent_instruction,
        model=model,          # Pass the loaded transformers model
        tokenizer=tokenizer   # Pass the loaded transformers tokenizer
    )

    # 3. Create a Runner instance
    runner = Runner(
        agent=medical_agent,
        session_service=session_service,
        artifact_service=artifact_service,
        app_name="MyMedicalApp"  # Add app_name here with a suitable name
    )

    # 4. Define a test question
    test_question = """while diminished ovarian reserve dor predicts decreased ovarian response to
stimulation it does not necessarily foretell about the fecundity cycle acco
rding to bolognas criteria laid down by the european society of human repro
duction and embryology old age abnormal ovarian reserve tests such as AFC a
fc and antimullerian hormone amh as well as prior suboptimal response to st
imulation are the main AF representing dor unfavorable response to maximal
stimulation on two previous occasions may also represent dor among the ovar
ian reserve tests amh and afc are the most predictive values for dor AF whi
ch may give rise to dor include environmental factors autoimmune or metabol
ic disorders infections genetic abnormalities and iatrogenic causes such as
smoking chemotherapy radiation and gynecologic surgeries besides studies ha
ve proposed endometriosis as a key contributor to dor and hence emphasized
on its proper management to prevent additional damages leading to compromis
ed fertility in summary dor is found to be a clinical challenge in the prac
tice of fertility care with controversial countermeasures to prevent or tre
at the condition nevertheless some promising measure such as oocyte embryo
and tissue cryopreservation ovarian transplantation dietary supplementation
and the transfer of mitochondria have offered hopes towards ameliorating th
e burden of dor this review attempts to discuss dor from different perspect
ives and summarize some existing hopes in clinical practice"""

    test_original_answer = "antral follicle count" # The expected answer for comparison

    print('\n')
    print(f"\n--- Testing the Agent (Basic Runner Call) ---")
    print(f"Input Question: {test_question}")
    print('\n')
    print(f"Expected Answer: {test_original_answer}")
    print('\n')

    # Use a unique user and session ID for this test
    user_id = "test_user"
    session_id = "test_session"

    try:
        # Execute the agent with user_id and session_id only
        run_result = await runner.run(user_id=user_id, session_id=session_id)
        async for event in run_result:
            if event.is_user_query():
                # If it expects a response, send the test question
                await runner.respond(user_id=user_id, session_id=session_id, response=test_question)
            if event.is_final_response():
                print(f"Agent Generated Answer: {event.message.text}")

    except Exception as e:
        print(f"An error occurred during agent execution: {e}")

In [ ]:
# Entry point to run the asynchronous example
if __name__ == "__main__":
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())




--- Testing the Agent (Basic Runner Call) ---
Input Question: while diminished ovarian reserve dor predicts decreased ovarian response to
stimulation it does not necessarily foretell about the fecundity cycle acco
rding to bolognas criteria laid down by the european society of human repro
duction and embryology old age abnormal ovarian reserve tests such as AFC a
fc and antimullerian hormone amh as well as prior suboptimal response to st
imulation are the main AF representing dor unfavorable response to maximal
stimulation on two previous occasions may also represent dor among the ovar
ian reserve tests amh and afc are the most predictive values for dor AF whi
ch may give rise to dor include environmental factors autoimmune or metabol
ic disorders infections genetic abnormalities and iatrogenic causes such as
smoking chemotherapy radiation and gynecologic surgeries besides studies ha
ve proposed endometriosis as a key contributor to dor and hence emphasized
on its proper management 